# Neural Network to predict Solar output

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd



In [5]:
path = '/Users/ben/Desktop/Senior-project/Final_Merged_Energy_and_Weather_Data.csv'
data = pd.read_csv(path)
 

In [6]:
features = ['cloudcover', 'uvindex', 'visibility', 'sealevelpressure', 'solarradiation']
clean_data = data.dropna(subset=features + ['solarenergy'])

X = clean_data[features]
y = clean_data['solarenergy']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [7]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)
test_loss = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"loss: {test_loss}")

Epoch 1/20
162/162 [==============================] - 1s 3ms/step - loss: 0.0757 - val_loss: 0.0035
Epoch 2/20
162/162 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0018
Epoch 3/20
162/162 [==============================] - 0s 2ms/step - loss: 0.0016 - val_loss: 0.0011
Epoch 4/20
162/162 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 8.6042e-04
Epoch 5/20
162/162 [==============================] - 0s 2ms/step - loss: 8.3775e-04 - val_loss: 7.6770e-04
Epoch 6/20
162/162 [==============================] - 0s 2ms/step - loss: 7.0233e-04 - val_loss: 6.4687e-04
Epoch 7/20
162/162 [==============================] - 0s 2ms/step - loss: 6.2718e-04 - val_loss: 6.4930e-04
Epoch 8/20
162/162 [==============================] - 0s 2ms/step - loss: 5.7242e-04 - val_loss: 5.8183e-04
Epoch 9/20
162/162 [==============================] - 0s 3ms/step - loss: 5.4942e-04 - val_loss: 4.9644e-04
Epoch 10/20
162/162 [==============================] - 0